In [ ]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import argparse
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
from torch.utils.data.sampler import SubsetRandomSampler
# import EarlyStopping
from pytorchtools2 import EarlyStopping
from  torch.optim.lr_scheduler import StepLR
from torch.optim import lr_scheduler
from torch.autograd import Variable
from transformers import BertForSequenceClassification
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile

use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [ ]:
data_dir = "./IDRID_Croped/" #---84%
#data_dir = "./IDRID_Croped_Augmented/"

# Number of classes in the dataset
TRAIN = 'train'
VAL = 'val'
TEST = 'test'
num_classes = 3 #len(class_names)

# Batch size for training (change depending on how much memory you have)
batch_size = 25

# Number of epochs to train for
num_epochs = 25
lr=0.01
step_size=2000
#Early stopping patience
patience = 10

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True


In [ ]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs=10):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    
    train_batches = len(dataloaders[TRAIN])
    val_batches = len(dataloaders[VAL])
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg.train(True)
        
        for i, data in enumerate(dataloaders[TRAIN]):
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches / 2), end='', flush=True)
                
            # Use half training dataset
            if i >= train_batches / 2:
                break
                
            inputs, labels = data
            
            if use_gpu:
#                 inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                inputs, labels = Variable(inputs), Variable(labels)
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            
            optimizer.zero_grad()  
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            #scheduler.step()
            
             #loss_train += loss.data[0]------------------
            loss_train += loss.item()
            acc_train += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
#             torch.cuda.empty_cache()
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train * 2 / dataset_sizes[TRAIN]
        avg_acc = acc_train * 2 / dataset_sizes[TRAIN]
        
        vgg.train(False)
        vgg.eval()
            
        for i, data in enumerate(dataloaders[VAL]):
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
                
            inputs, labels = data
            
            if use_gpu:
#                 inputs, labels = Variable(inputs.cuda(), volatile=True), Variable(labels.cuda(), volatile=True)
                inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
            else:
                inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.item()
            acc_val += torch.sum(preds == labels.data)
            
            del inputs, labels, outputs, preds
#             torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / dataset_sizes[VAL]
        avg_acc_val = acc_val / dataset_sizes[VAL]
        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    
    vgg.load_state_dict(best_model_wts)
    return vgg

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [ ]:

data_transforms = {
    TRAIN: transforms.Compose([
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ]),
    TEST: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
    
}


# ------------------------------------------------------
image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x), 
        transform=data_transforms[x]
    )
    for x in [TRAIN, VAL, TEST]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=batch_size,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, VAL, TEST]
}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL, TEST]}

for x in [TRAIN, VAL, TEST]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
class_names = image_datasets[TRAIN].classes
print(image_datasets[TRAIN].classes)
# ------------------------------------------------------


In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    # plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes = next(iter(dataloaders[TRAIN]))
show_databatch(inputs, classes)

In [ ]:
def visualize_model(vgg, num_images=6):
    was_training = vgg.training
    
    # Set model for evaluation
    vgg.train(False)
    vgg.eval() 
    
    images_so_far = 0

    for i, data in enumerate(dataloaders[TEST]):
        inputs, labels = data
        size = inputs.size()[0]
        
        if use_gpu:
#             inputs, labels = Variable(inputs.device(), volatile=True), Variable(labels.device(), volatile=True)
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        else:
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        
        outputs = vgg(inputs)
        
        _, preds = torch.max(outputs.data, 1)
        predicted_labels = [preds[j] for j in range(inputs.size()[0])]
        
        print("Ground truth:")
        show_databatch(inputs.data.cpu(), labels.data.cpu())
        print("Prediction:")
        show_databatch(inputs.data.cpu(), predicted_labels)
        
        del inputs, labels, outputs, preds, predicted_labels
#         torch.cuda.empty_cache()
        
        images_so_far += size
        if images_so_far >= num_images:
            break
        
    vgg.train(mode=was_training) # Revert model back to original training state

In [ ]:
def eval_model(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(dataloaders[TEST])
    print("Evaluating model")
    print('-' * 10)
    
    for i, data in enumerate(dataloaders[TEST]):
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)

        vgg.train(False)
        vgg.eval()
        inputs, labels = data

        if use_gpu:
#             inputs, labels = Variable(inputs.to(device), volatile=True), Variable(labels.to(device), volatile=True)
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        else:
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)

        outputs = vgg(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.item()
        acc_test += torch.sum(preds == labels.data)

        del inputs, labels, outputs, preds
#         torch.cuda.empty_cache()
        
    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)

In [ ]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18 ,Resnet34
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
#         model_ft = torchvision.models.quantization.resnet18(pretrained=True, quantize=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
       
        

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224
        
    elif model_name == "EfficientNet":
        """ EfficientDet-d1,EfficientNet-b1 
        """
        model_ft = models.efficientnet_b1(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        #num_ftrs = model_ft.classifier.in_features
        num_ftrs = model_ft.classifier[1].in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        

    elif model_name == "mobilenet":
       
        """ mobilenet ,mobilenet
        """
        model_ft = torchvision.models.quantization.mobilenet_v2(pretrained=True, quantize=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        #num_ftrs = model_ft.classifier.in_features
        num_ftrs = model_ft.classifier[1].in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224
        
        
    
    elif model_name == "vgg":
        """ VGG11_bn ,VGG16_bn
        """
                
#         model_ft = torchvision.models.quantization.vgg11_bn(pretrained=True, quantize=True)
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
         
        print(model_ft.classifier[6].out_features) # 1000 --------------------------
        # Freeze training for all layers
        for param in model_ft.features.parameters():
             param.require_grad = False
        
        # Newly created modules have require_grad=True by default
        num_ftrs = model_ft.classifier[6].in_features
        features = list(model_ft.classifier.children())[:-1] # Remove last layer
        features.extend([nn.Linear(num_ftrs, len(class_names))]) # Add our layer with 4 outputs
        model_ft.classifier = nn.Sequential(*features) # Replace the model classifier
        print(model_ft)
        
    
    
         #set_parameter_requires_grad(model_ft, feature_extract)
        #num_ftrs = model_ft.classifier[6].in_features
        #model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet,Squeezenet_1
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet,121,161
        """
        model_ft = models.densenet161(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size


# # Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
# model_name = "vgg"

# # Initialize the model for this run
# # #/////////////////////////////////////////////////////
# model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# # # Print the model we just instantiated
# print(model_ft)

In [ ]:
# Load the pretrained model from pytorch
model_ft = models.vgg16_bn()
model_ft.load_state_dict(torch.load("./pretain/vgg/vgg16_bn.pth"))

# r18.load_state_dict(torch.load(PATH))  # load the saved model
     # We now have an instance of the pretrained model




print(model_ft.classifier[6].out_features) # 1000 


# Freeze training for all layers
for param in model_ft.features.parameters():
    param.require_grad = False

# Newly created modules have require_grad=True by default
num_features = model_ft.classifier[6].in_features
features = list(model_ft.classifier.children())[:-1] # Remove last layer
features.extend([nn.Linear(num_features, len(class_names))]) # Add our layer with  outputs
model_ft.classifier = nn.Sequential(*features) # Replace the model classifier



print(model_ft)

In [ ]:
# If you want to train the model for more than 2 epochs, set this to True after the first run
resume_training = False

if resume_training:  
    print("Loading pretrained model..")
        #model_ft.load_state_dict(torch.load('./pretain/resnet50-19c8e357.pth'))
    
    checkpoint  = torch.load('./pretain/output/av 0.8350  best 0.8544 .pt', map_location=torch.device('cpu'))
    model_ft = torch.nn.DataParallel(model_ft )
    model_ft.load_state_dict(checkpoint, strict=False)

    #print("Loaded!")
 

    #model_ft.load_state_dict(torch.load('./pretain/vgg/VGG16_v2-OCT_Retina_half_dataset.pt'))
   

    #model_ft = BertForSequenceClassification.from_pretrained("./pretain/vgg/VGG16_v2-OCT_Retina_half_dataset.pt", num_labels= 3, ignore_mismatched_sizes=True)
    

   
  
    print("Loaded!")
    
    


In [ ]:
def imshowxxxxxx(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
# inputs, classes = next(iter(dataloaders_dict['train']))

# Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [ ]:
# Send the model to GPU
#model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

            
# Observe that all parameters are being optimized lr=0.001, test 0.01,0.0001
#optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
#optimizer_ft = optim.SGD(params_to_update, lr=0.1)# 

# if use_gpu:
#     model_ft.cuda() #.cuda() will move everything to the GPU side
device = torch.device("cpu")
model_ft.to(device)
criterion = nn.CrossEntropyLoss()

optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=step_size, gamma=0.1)




In [ ]:
def eval_modelxxxxxxxxx(vgg, criterion):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    
    test_batches = len(dataloaders[TEST])
    print("Evaluating model")
    print('-' * 10)
    
    for i, data in enumerate(dataloaders[TEST]):
        if i % 100 == 0:
            print("\rTest batch {}/{}".format(i, test_batches), end='', flush=True)

        vgg.train(False)
        vgg.eval()
        inputs, labels = data

        if use_gpu:
#             inputs, labels = Variable(inputs.device(), volatile=True), Variable(labels.device(), volatile=True)
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        else:
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)

        outputs = vgg(inputs)

        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        loss_test += loss.item()
        acc_test += torch.sum(preds == labels.data)

        del inputs, labels, outputs, preds
        torch.device.empty_cache()
        
    avg_loss = loss_test / dataset_sizes[TEST]
    avg_acc = acc_test / dataset_sizes[TEST]
    
    elapsed_time = time.time() - since
    print()
    print("Evaluation completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Avg loss (test): {:.4f}".format(avg_loss))
    print("Avg acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)  

In [ ]:
print("Test before training")
# eval_model(model_ft, criterion)

In [ ]:
# visualize_model(model_ft)
from torch.utils.mobile_optimizer import optimize_for_mobile
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile
from torchvision.models.vgg import vgg16

In [ ]:
def print_size_of_model(model, label=""):
    torch.save(model.state_dict(), "temp.p")
    size=os.path.getsize("temp.p")
    print("model: ",label,' \t','Size (KB):', size/1e3)
    os.remove('temp.p')
    return size

In [ ]:
# Setup the loss fxn
# criterion = nn.CrossEntropyLoss()

# Train and evaluate
#model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, 
                             #patience=patience, is_inception=(model_name=="inception"))

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs)
# torch.save(model_ft.state_dict(), './pretain/output/VGG16_v2-DME.pt')


model_int8 = torch.quantization.quantize_dynamic(model_ft,  # the original model
                                                  {nn.LSTM,nn.Linear},  # a set of layers to dynamically quantize
                                                  dtype=torch.qint8)
    
f=print_size_of_model(model_ft,"model_fp32")
q=print_size_of_model(model_int8,"model_int8")
print("{0:.2f} times smaller".format(f/q))




scripted_module = torch.jit.script(model_int8)
optimized_scripted_module = optimize_for_mobile(scripted_module)

torch.jit.save(optimized_scripted_module, "mobilemodel.pt")




In [ ]:
eval_model(model_ft, criterion)

In [ ]:
visualize_model(model_ft, num_images=3)

In [ ]:
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def confusionMatrix(vgg, num_images):
    was_training = vgg.training
    
    # Set model for evaluation
    vgg.train(False)
    vgg.eval() 
    
    images_so_far = 0
  
        
        
    for i, data in enumerate(dataloaders[TEST]):
        inputs, labels = data
        size = inputs.size()[0]
        
        if use_gpu:
#             inputs, labels = Variable(inputs.device(), volatile=True), Variable(labels.device(), volatile=True)
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        else:
            inputs, labels = Variable(inputs, volatile=True), Variable(labels, volatile=True)
        
        outputs = vgg(inputs)
        
        _, preds = torch.max(outputs.data, 1)
        predicted_labels = [preds[j] for j in range(inputs.size()[0])]
        
        
        
        print("Ground truth:")
        show_databatch(inputs.data.cpu(), labels.data.cpu())
        print("Prediction:")
        show_databatch(inputs.data.cpu(), predicted_labels)
  
        #print(metrics.confusion_matrix(labels.data.cpu(),preds.data.cpu()));    
       
        print(metrics.classification_report(labels.data.cpu(),preds.data.cpu(),digits=3));


        cm1 = confusion_matrix(labels.data.cpu(),preds.data.cpu());
        print('Confusion Matrix : \n', cm1)
        
        total1=sum(sum(cm1))
#####from confusion matrix calculate accuracy
        accuracy1=(cm1[0,0]+cm1[1,1]+cm1[2,2])/total1
        print ('Accuracy : ', accuracy1)
#         format_float = "{:.2f}".format(accuracy1)
#         print ('Accuracy : ', format_float)

        print('Specificity: -macro: ', specificity_score(labels.data.cpu(),preds.data.cpu(), average='macro'))
        print('Sensitivity: -macro : ', sensitivity_score(labels.data.cpu(),preds.data.cpu(), average='macro'))
        print('precision_score: -macro', precision_score(labels.data.cpu(),preds.data.cpu(), average='macro'))
        print('f1_score: -macro', f1_score(labels.data.cpu(),preds.data.cpu(), average='macro'))
        
        print('Specificity: -weighted: ', specificity_score(labels.data.cpu(),preds.data.cpu(), average='weighted'))
        print('Sensitivity: -weighted : ', sensitivity_score(labels.data.cpu(),preds.data.cpu(), average='weighted'))
        print('f1_score: -weighted', f1_score(labels.data.cpu(),preds.data.cpu(), average='weighted'))
        print('precision_score: -weighted', precision_score(labels.data.cpu(),preds.data.cpu(), average='weighted'))
        
        print('matthews_corrcoef: ', matthews_corrcoef(labels.data.cpu(),preds.data.cpu()))
        print('accuracy_score: ', accuracy_score(labels.data.cpu(),preds.data.cpu(), normalize=True))

        
        
        cm = metrics.confusion_matrix(labels.data.cpu(),preds.data.cpu())
        plot_confusion_matrix(cm, classes=['True', 'False'])

        del inputs, labels, outputs, preds, predicted_labels
        
        
        
#         torch.device.empty_cache()
        
        images_so_far += size
        if images_so_far >= num_images:
            break
        
    vgg.train(mode=was_training) # Revert model back to original training state

In [ ]:
import sklearn.metrics as metrics
import sklearn.metrics as confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import itertools
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
from imblearn.metrics import specificity_score
from imblearn.metrics import sensitivity_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import balanced_accuracy_score

confusionMatrix(model_ft, num_images=12)

testing_datagen = ImageDataGenerator(rescale=1. / 255)
testing_generator = testing_datagen.flow_from_directory(
testing_dir,
target_size=(200, 200),
batch_size=batch_size,
shuffle=False,
class_mode='categorical')